In [2]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [4]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [5]:
tqdm_pandas(tqdm)

### Работа с данными (1 балл)

Загрузите датасет, с которым вы работали во время соревнования на kaggle. Преобразуйте его в формат, удобный для обучения модели. В качетсве фичей используйте эмбединги слов.

In [6]:
mystem = Mystem()

In [7]:
upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}

In [8]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [10]:
def text_prep_tags(text):
    
    text = text.lower()
    result = []
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
    
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            result.append(token)

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    
    return result

In [18]:

df_train = pd.read_csv('train.csv',sep=',')

In [12]:
# http://vectors.nlpl.eu/repository/11/185.zip
word2vec = KeyedVectors.load_word2vec_format('../w2v/tayga_upos_skipgram_300_2_2019.bin', binary=True)

In [132]:
#word2vec.vocab

In [19]:
count_v = CountVectorizer(tokenizer=text_prep_tags)
count_v.fit(tqdm_notebook(df_train['text']))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function text_prep_tags at 0x7f8bf2435730>,
        vocabulary=None)

In [20]:
token_text = df_train['text'].progress_apply(text_prep_tags)
#X_test['text'] = X_test['text'].progress_apply(text_prep_tags)


100%|██████████| 112467/112467 [00:44<00:00, 2543.38it/s]


In [21]:
train_ids = token_text.progress_apply(lambda x: list(map(lambda y: count_v.vocabulary_[y] +1 , x)))

100%|██████████| 112467/112467 [00:00<00:00, 230251.13it/s]


In [42]:
X_train_ids,X_val_ids,y_train,y_val = train_test_split(train_ids,X_train['label'],test_size=0.3)

In [43]:
X_train_ids.shape

(78726,)

In [8]:
X_train_ids_pad


NameError: name 'X_train_ids_pad' is not defined

In [44]:
vocab_size = len(count_v.vocabulary_)
sentence_size = 20
embedding_size = 300


# первые индексы в словаре зарезервированы за специальными токенами 
# для паддинга, начала предложения, и для слов которые не вошли в словарь
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2



print("Pad sequences (samples x time)")
X_train_ids_pad = sequence.pad_sequences(X_train_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", X_train_ids.shape)
X_val_ids_pad = sequence.pad_sequences(X_val_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", X_val_ids.shape)


Pad sequences (samples x time)
x_train shape: (78726,)
x_train shape: (33741,)


In [45]:
def initializer(shape=None, dtype=None, partition_info=None):    
    vocab_dict = count_v.vocabulary_
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size+1, embedding_size))
    num_loaded = 0
    for w, i in vocab_dict.items():
        v = None
        try:
            v = word2vec[w]
        except KeyError: # не нашли такой токен в словаре
                pass
        if v is not None :
            embedding_matrix[i+1] = v
            num_loaded += 1
   
    embedding_matrix = embedding_matrix.astype(np.float32)
    embedding_matrix[0] = np.zeros(300)
    return embedding_matrix

In [49]:
np.max(x_len_train)

626

In [38]:
model_dir = './model'

In [47]:
x_len_train = np.array([len(x) for x in X_train_ids])
x_len_test = np.array([len(x) for x in X_val_ids])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_train_ids_pad, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(X_train_ids_pad))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_val_ids_pad, x_len_test, y_val))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [59]:
all_classifiers = {}
def train_and_evaluate(net,net_name,n_step):
    params = {'net':net}
    classifier = tf.estimator.Estimator(model_fn=model_fn,
                                         model_dir=os.path.join(model_dir, net_name), params=params)
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=n_step)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['class_ids'] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes

#     pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
#     with tf.Session() as sess:
#         writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
#         writer.add_summary(sess.run(pr), global_step=0)
#         writer.close()

In [53]:
def model_fn(features, labels, mode, params):  
    
    # Compute predictions.
    net = params['net']
    logits = net(features)
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')
    f1 = tf_metrics.f1(labels=labels,
                               predictions=predicted_classes,num_classes=3,average='micro')
    
    metrics = {'accuracy': accuracy,'f1':f1}
    tf.summary.scalar('accuracy', accuracy[1])
    
    # Compute evaluation
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [49]:
# classifier = tf.estimator.DNNClassifier(
#     hidden_units=[100],
#     feature_columns=[column], 
#     model_dir=os.path.join(model_dir, 'bow_embeddings'))
# train_and_evaluate(classifier)

### CNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- Dense

Параметры модели подберите сами.

In [55]:
def cnn_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    conv = tf.layers.conv1d(
        inputs=inputs,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    print(conv.get_shape())
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    print(pool.get_shape())
    #hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=pool, units=3)

    return logits

In [85]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()

In [133]:
%time
train_and_evaluate(cnn_fn,'cnn',15000)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.44 µs
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bc81c6828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(?, 20, 32)
(?, 32)
INFO:tensorflow:Done cal

INFO:tensorflow:loss = 0.14323622, step = 7001 (0.376 sec)
INFO:tensorflow:global_step/sec: 216.221
INFO:tensorflow:loss = 0.10346129, step = 7101 (0.463 sec)
INFO:tensorflow:global_step/sec: 260.724
INFO:tensorflow:loss = 0.13817972, step = 7201 (0.384 sec)
INFO:tensorflow:global_step/sec: 272.374
INFO:tensorflow:loss = 0.1559169, step = 7301 (0.367 sec)
INFO:tensorflow:global_step/sec: 261.618
INFO:tensorflow:loss = 0.16318744, step = 7401 (0.382 sec)
INFO:tensorflow:global_step/sec: 267.079
INFO:tensorflow:loss = 0.19950533, step = 7501 (0.374 sec)
INFO:tensorflow:global_step/sec: 269.482
INFO:tensorflow:loss = 0.16274379, step = 7601 (0.371 sec)
INFO:tensorflow:global_step/sec: 262.461
INFO:tensorflow:loss = 0.13285883, step = 7701 (0.381 sec)
INFO:tensorflow:global_step/sec: 264.245
INFO:tensorflow:loss = 0.18393986, step = 7801 (0.378 sec)
INFO:tensorflow:global_step/sec: 216.871
INFO:tensorflow:loss = 0.22563109, step = 7901 (0.461 sec)
INFO:tensorflow:global_step/sec: 264.467
I

INFO:tensorflow:Starting evaluation at 2019-03-23T20:35:26Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/cnn/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-23-20:35:28
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.8048665, f1 = 0.80486643, global_step = 15000, loss = 0.7661977
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: ./model/cnn/model.ckpt-15000
INFO:tensorflow:Calling model_fn.
(?, 20, 32)
(?, 32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/cnn/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


### RNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- RNN
- Dense

Параметры модели подберите сами.

In [60]:
def rnn_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)
    print(inputs.get_shape())
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=10)
    output, state = tf.nn.dynamic_rnn(cell, inputs,sequence_length=features['len'], dtype=tf.float32)
    print(state.get_shape())
    logits = tf.layers.dense(inputs=state, units=3)

    return logits

In [62]:
%time
train_and_evaluate(rnn_fn,'rnn',15000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bae52acc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(?, 20, 300)
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be r

INFO:tensorflow:loss = 0.42576203, step = 6501 (0.372 sec)
INFO:tensorflow:global_step/sec: 261.584
INFO:tensorflow:loss = 0.4385354, step = 6601 (0.382 sec)
INFO:tensorflow:global_step/sec: 262.321
INFO:tensorflow:loss = 0.36432162, step = 6701 (0.381 sec)
INFO:tensorflow:global_step/sec: 247.607
INFO:tensorflow:loss = 0.44337943, step = 6801 (0.404 sec)
INFO:tensorflow:global_step/sec: 256.642
INFO:tensorflow:loss = 0.4480545, step = 6901 (0.390 sec)
INFO:tensorflow:global_step/sec: 270.456
INFO:tensorflow:loss = 0.37016696, step = 7001 (0.370 sec)
INFO:tensorflow:global_step/sec: 220.679
INFO:tensorflow:loss = 0.35342625, step = 7101 (0.453 sec)
INFO:tensorflow:global_step/sec: 229.327
INFO:tensorflow:loss = 0.42997023, step = 7201 (0.436 sec)
INFO:tensorflow:global_step/sec: 230.762
INFO:tensorflow:loss = 0.3186023, step = 7301 (0.433 sec)
INFO:tensorflow:global_step/sec: 240.411
INFO:tensorflow:loss = 0.47367692, step = 7401 (0.416 sec)
INFO:tensorflow:global_step/sec: 224.001
INF

INFO:tensorflow:loss = 0.3551077, step = 14701 (0.363 sec)
INFO:tensorflow:global_step/sec: 274.952
INFO:tensorflow:loss = 0.37444222, step = 14801 (0.364 sec)
INFO:tensorflow:global_step/sec: 274.921
INFO:tensorflow:loss = 0.35757232, step = 14901 (0.364 sec)
INFO:tensorflow:Saving checkpoints for 15000 into ./model/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.3899919.
INFO:tensorflow:Calling model_fn.
(?, 20, 300)
(?, 10)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-23T17:15:23Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/rnn/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-23-17:15:25
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.814143, f1 = 0.81414294, global_step = 15000, loss = 0.42650166
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: ./model/rnn/m

### LSTM model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- LSTM
- Dense

Параметры модели подберите сами.

In [63]:
def lstm_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)

    # create the complete LSTM
    _, final_states = tf.nn.dynamic_rnn(
        lstm_cell, inputs, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states.h
    
    logits = tf.layers.dense(inputs=outputs, units=3)

    return logits

In [64]:
%time
train_and_evaluate(lstm_fn,'lstm',15000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/lstm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8b8fbae828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that i

INFO:tensorflow:loss = 0.15990573, step = 6801 (1.602 sec)
INFO:tensorflow:global_step/sec: 62.7755
INFO:tensorflow:loss = 0.27900618, step = 6901 (1.593 sec)
INFO:tensorflow:global_step/sec: 63.4165
INFO:tensorflow:loss = 0.2586225, step = 7001 (1.577 sec)
INFO:tensorflow:global_step/sec: 59.6973
INFO:tensorflow:loss = 0.18228342, step = 7101 (1.675 sec)
INFO:tensorflow:global_step/sec: 62.4789
INFO:tensorflow:loss = 0.10987206, step = 7201 (1.600 sec)
INFO:tensorflow:global_step/sec: 62.2019
INFO:tensorflow:loss = 0.25753725, step = 7301 (1.608 sec)
INFO:tensorflow:global_step/sec: 62.4934
INFO:tensorflow:loss = 0.1933045, step = 7401 (1.600 sec)
INFO:tensorflow:global_step/sec: 60.6853
INFO:tensorflow:loss = 0.19036193, step = 7501 (1.648 sec)
INFO:tensorflow:global_step/sec: 62.9003
INFO:tensorflow:loss = 0.14388603, step = 7601 (1.590 sec)
INFO:tensorflow:global_step/sec: 61.2018
INFO:tensorflow:loss = 0.19624723, step = 7701 (1.634 sec)
INFO:tensorflow:global_step/sec: 61.741
INF

INFO:tensorflow:Saving checkpoints for 15000 into ./model/lstm/model.ckpt.
INFO:tensorflow:Loss for final step: 0.11224124.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-23T17:25:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/lstm/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-23-17:25:35
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.84923387, f1 = 0.84923387, global_step = 15000, loss = 0.49706423
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: ./model/lstm/model.ckpt-15000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/lstm/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init

Сравните все три реализованные модели по времени и по качеству классификации. Какая лучше? Как думаете, почему?

<b>Вывод:<br>
 - cnn работает быстрее всего, качество самое низкое <br>
 - rnn работает дольше cnn, качество немного лучше, за счет учета предыдущих слов в последовательности <br>
 - lstm работает заметно дольше предыдущих, качество получилось сильно лучше , видно потому что запоминалось больше информации из текста (а текст небольшой) <br>
</b>

### Baseline model (2 балла)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Параметры для Conv1D, MaxPooling1D и RNN подберите сами.

In [6]:
def baseline_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    conv = tf.layers.conv1d(
        inputs=inputs,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    print(conv.get_shape())
    conv = tf.print(conv)
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    print(pool.get_shape())
    pool = tf.expand_dims(pool, 1)
    print(pool.get_shape())
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=10)
    output, state = tf.nn.dynamic_rnn(cell, pool, sequence_length=features['len'], dtype=tf.float32)
    
    logits = tf.layers.dense(inputs=state, units=3)

    return logits

In [7]:
%time
train_and_evaluate(baseline_fn,'baseline',15000)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


NameError: name 'train_and_evaluate' is not defined

### Dropout (1 балл)

Добавьте dropout к baseline моделе. Пример модефицированной модели:
- Dropout
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Подберите параметры.

In [79]:
def drop_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)
    
    drop = tf.layers.dropout(inputs)
    conv = tf.layers.conv1d(
        inputs=drop,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    print(pool.get_shape())
    pool = tf.expand_dims(pool, 1)
    
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=10)
    output, state = tf.nn.dynamic_rnn(cell, pool, sequence_length=features['len'],dtype=tf.float32)
    
    logits = tf.layers.dense(inputs=state, units=3)

    return logits

In [80]:
%time
train_and_evaluate(drop_fn,'drop',15000)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/drop', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8bae308dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(?, 32)
INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:global_step/sec: 219.38
INFO:tensorflow:loss = 0.11669013, step = 7101 (0.456 sec)
INFO:tensorflow:global_step/sec: 265.723
INFO:tensorflow:loss = 0.15038598, step = 7201 (0.376 sec)
INFO:tensorflow:global_step/sec: 263.054
INFO:tensorflow:loss = 0.116947174, step = 7301 (0.380 sec)
INFO:tensorflow:global_step/sec: 267.426
INFO:tensorflow:loss = 0.16304371, step = 7401 (0.374 sec)
INFO:tensorflow:global_step/sec: 265.923
INFO:tensorflow:loss = 0.22834854, step = 7501 (0.376 sec)
INFO:tensorflow:global_step/sec: 266.298
INFO:tensorflow:loss = 0.13555762, step = 7601 (0.376 sec)
INFO:tensorflow:global_step/sec: 269.93
INFO:tensorflow:loss = 0.2603898, step = 7701 (0.370 sec)
INFO:tensorflow:global_step/sec: 266.996
INFO:tensorflow:loss = 0.26412705, step = 7801 (0.375 sec)
INFO:tensorflow:global_step/sec: 218.175
INFO:tensorflow:loss = 0.14806814, step = 7901 (0.458 sec)
INFO:tensorflow:global_step/sec: 267.978
INFO:tensorflow:loss = 0.09878707, step = 8001 (0.373 sec)
IN

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/drop/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-23-17:55:12
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.8019027, f1 = 0.80190265, global_step = 15000, loss = 0.81040287
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: ./model/drop/model.ckpt-15000
INFO:tensorflow:Calling model_fn.
(?, 32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/drop/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Улучшило ли результат классификации использование Dropout? Как думаете, почему?

<b>Вывод:<br> 
Улучшило, но незначительно. Улучшилось из-за большей обобщающей способности</b>

### Bidirectional (1 балл)

Вместо RNN в первой модели используйте biRNN. Пример модефецированной модели:
- Conv1D, activation='relu'
- MaxPooling1D
- biRNN
- Dense

Подберите параметры.

In [84]:
def birnn_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)
    
    
    conv = tf.layers.conv1d(
        inputs=inputs,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    print(conv.get_shape())
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    pool = tf.expand_dims(pool, 1)
    print(pool.get_shape())
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=10)
    output, state = tf.nn.bidirectional_dynamic_rnn(cell,cell, pool, dtype=tf.float32)
    
    print('outout',output[0].get_shape())
    state = tf.concat(state, 1)
    print(state.get_shape())
    
    logits = tf.layers.dense(inputs=state, units=3)

    return logits

In [85]:
%time
train_and_evaluate(birnn_fn,'birnn',15000)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.1 µs
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/birnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8b9071e908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(?, 20, 32)
(?, 1, 32)
outout (?, 1, 10)
(?, 2

INFO:tensorflow:global_step/sec: 258.901
INFO:tensorflow:loss = 0.24698015, step = 7001 (0.386 sec)
INFO:tensorflow:global_step/sec: 216.657
INFO:tensorflow:loss = 0.06469615, step = 7101 (0.462 sec)
INFO:tensorflow:global_step/sec: 252.299
INFO:tensorflow:loss = 0.13757798, step = 7201 (0.396 sec)
INFO:tensorflow:global_step/sec: 237.417
INFO:tensorflow:loss = 0.23003517, step = 7301 (0.421 sec)
INFO:tensorflow:global_step/sec: 262.371
INFO:tensorflow:loss = 0.23760073, step = 7401 (0.381 sec)
INFO:tensorflow:global_step/sec: 263.042
INFO:tensorflow:loss = 0.15024517, step = 7501 (0.380 sec)
INFO:tensorflow:global_step/sec: 258.667
INFO:tensorflow:loss = 0.23179212, step = 7601 (0.387 sec)
INFO:tensorflow:global_step/sec: 266.25
INFO:tensorflow:loss = 0.15435717, step = 7701 (0.376 sec)
INFO:tensorflow:global_step/sec: 260.936
INFO:tensorflow:loss = 0.1541626, step = 7801 (0.383 sec)
INFO:tensorflow:global_step/sec: 218.5
INFO:tensorflow:loss = 0.11679504, step = 7901 (0.458 sec)
INFO

(?, 20, 32)
(?, 1, 32)
outout (?, 1, 10)
(?, 20)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-23T18:01:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/birnn/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-23-18:01:22
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.8042145, f1 = 0.8042145, global_step = 15000, loss = 0.8218594
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: ./model/birnn/model.ckpt-15000
INFO:tensorflow:Calling model_fn.
(?, 20, 32)
(?, 1, 32)
outout (?, 1, 10)
(?, 20)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/birnn/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Дало ли буст использовании biRNN вместо RNN? Сильный? Как думаете, почему?

<b>Вывод:<br>
Буста нет. Так как текст очень короткий, то birnn не имеют смысла и не раскрывают свой потенциал.</b>

### Custom model (2 балла)

Подберите архитектуру сети, используя сверточные слои и слои макспулинга и превзойдите качество полносвязной сети.

Подберите архитектуру сети, используя следующие типы слоев:
- Conv1D
- MaxPooling1D
- RNN
- LSTM
- GRU
- Dropout
- Dense

Можно использовать любые из описанных слоев в любом количестве и в любом порядке. Не обязательно использовать все описанные слои. 

Настройте параметры. Превзойдите лучшее качество, полученное раннее. 

In [130]:
def custom_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)
    print(inputs.get_shape())
    
    conv = tf.layers.conv1d(
        inputs=inputs,
        filters=200,
        kernel_size=3,
        #strides=2,
        padding="same",
        activation=tf.nn.relu)
    print(conv.get_shape())
    
#     conv2 = tf.layers.conv1d(
#         inputs=conv,
#         filters=50,
#         kernel_size=3,
#         padding="same",
#         activation=tf.nn.relu)
#     print(conv2.get_shape())

    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=20)
    output, state = tf.nn.dynamic_rnn(cell, conv, sequence_length=features['len'],dtype=tf.float32)
    
    #pool = tf.reshape(conv2,(-1,1000))
    # Global Max Pooling
    #pool = tf.reduce_max(input_tensor=conv2, axis=1)
   #print(pool.get_shape())
    #pool = tf.expand_dims(pool, 1)
    drop1 = tf.layers.dropout(state)
    d1 = tf.layers.dense(inputs=drop1, units=300)
    drop2 = tf.layers.dropout(d1)
    logits = tf.layers.dense(inputs=drop2, units=3)

    return logits

In [131]:
%time
train_and_evaluate(custom_fn,'custom',25000)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/custom', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8b97ad6be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
(?, 20, 300)
(?, 20, 200)
INFO:tensorflow:Don

INFO:tensorflow:loss = 0.10180237, step = 7001 (1.086 sec)
INFO:tensorflow:global_step/sec: 82.9982
INFO:tensorflow:loss = 0.1829673, step = 7101 (1.205 sec)
INFO:tensorflow:global_step/sec: 92.3969
INFO:tensorflow:loss = 0.03799048, step = 7201 (1.082 sec)
INFO:tensorflow:global_step/sec: 92.4623
INFO:tensorflow:loss = 0.0338689, step = 7301 (1.082 sec)
INFO:tensorflow:global_step/sec: 91.317
INFO:tensorflow:loss = 0.06841419, step = 7401 (1.095 sec)
INFO:tensorflow:global_step/sec: 91.746
INFO:tensorflow:loss = 0.13923335, step = 7501 (1.090 sec)
INFO:tensorflow:global_step/sec: 90.6955
INFO:tensorflow:loss = 0.35177433, step = 7601 (1.102 sec)
INFO:tensorflow:global_step/sec: 90.7594
INFO:tensorflow:loss = 0.18653987, step = 7701 (1.102 sec)
INFO:tensorflow:global_step/sec: 90.1749
INFO:tensorflow:loss = 0.1571298, step = 7801 (1.109 sec)
INFO:tensorflow:global_step/sec: 84.3253
INFO:tensorflow:loss = 0.06531294, step = 7901 (1.186 sec)
INFO:tensorflow:global_step/sec: 89.1021
INFO:

INFO:tensorflow:global_step/sec: 90.3873
INFO:tensorflow:loss = 0.008776536, step = 15201 (1.106 sec)
INFO:tensorflow:global_step/sec: 88.6109
INFO:tensorflow:loss = 0.009343293, step = 15301 (1.129 sec)
INFO:tensorflow:global_step/sec: 90.4551
INFO:tensorflow:loss = 0.045746222, step = 15401 (1.105 sec)
INFO:tensorflow:global_step/sec: 91.5128
INFO:tensorflow:loss = 0.005584297, step = 15501 (1.093 sec)
INFO:tensorflow:global_step/sec: 89.9806
INFO:tensorflow:loss = 0.02311113, step = 15601 (1.111 sec)
INFO:tensorflow:global_step/sec: 90.3837
INFO:tensorflow:loss = 0.010776249, step = 15701 (1.106 sec)
INFO:tensorflow:global_step/sec: 85.1916
INFO:tensorflow:loss = 0.009990138, step = 15801 (1.174 sec)
INFO:tensorflow:global_step/sec: 90.5257
INFO:tensorflow:loss = 0.030181479, step = 15901 (1.105 sec)
INFO:tensorflow:global_step/sec: 89.426
INFO:tensorflow:loss = 0.010113623, step = 16001 (1.118 sec)
INFO:tensorflow:global_step/sec: 89.3882
INFO:tensorflow:loss = 0.018447237, step = 

INFO:tensorflow:global_step/sec: 90.2541
INFO:tensorflow:loss = 0.03478628, step = 23301 (1.108 sec)
INFO:tensorflow:global_step/sec: 90.0123
INFO:tensorflow:loss = 0.004523314, step = 23401 (1.111 sec)
INFO:tensorflow:global_step/sec: 89.8817
INFO:tensorflow:loss = 0.04055793, step = 23501 (1.113 sec)
INFO:tensorflow:global_step/sec: 91.2912
INFO:tensorflow:loss = 0.009288075, step = 23601 (1.096 sec)
INFO:tensorflow:global_step/sec: 84.7121
INFO:tensorflow:loss = 0.0036593955, step = 23701 (1.180 sec)
INFO:tensorflow:global_step/sec: 89.7854
INFO:tensorflow:loss = 0.0064744037, step = 23801 (1.114 sec)
INFO:tensorflow:global_step/sec: 89.7962
INFO:tensorflow:loss = 0.0016156211, step = 23901 (1.114 sec)
INFO:tensorflow:global_step/sec: 90.1199
INFO:tensorflow:loss = 0.0021503144, step = 24001 (1.110 sec)
INFO:tensorflow:global_step/sec: 90.0159
INFO:tensorflow:loss = 0.014009464, step = 24101 (1.111 sec)
INFO:tensorflow:global_step/sec: 90.4966
INFO:tensorflow:loss = 0.029574964, ste